In [ ]:
import sklearn as sk
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split #Обращение к этой функции через алиас sk - не получается. Почему?
from subprocess import check_output
#_________________________ ЧТЕНИЕ ИЗ ФАЙЛА  ____________________________
data = pd.read_csv('../input/train.csv')
#X_Fin_test = pd.read_csv('../input/test.csv')
#ids = X_Fin_test['id']
#X_Fin_test.drop(['id'], axis=1)

#print(data.head())
print(data.shape)
print(data[:1])
#print(data.iloc[:3, 1])
print(data.describe()) # сводная инфа по таблице
# predict a value for both formation_energy_ev_natom and bandgap_energy_ev
dataX = data.copy().iloc[:, 1: 12]  # без колонки id и последних двух
#dataX.drop(['spacegroup'], axis=1, inplace=True)
dataY = data.copy().iloc[:, 12: ]   # последний 1 (или 2) столбца


In [ ]:
#____________________ ОЦЕНКА ГРАФИКАМИ  _____________________

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style= 'whitegrid', context ='notebook')
#cols = [ 'LSTAT', 'INDUS', 'NOX','RМ','MEDV']
sns.pairplot(data.iloc[:, :], size=10)
plt.show()

In [ ]:
sns.distplot(data.iloc[:, 12:13])
plt.show()
sns.distplot(data.iloc[:, 13:])
plt.show()

In [ ]:
# Show the results of a 
sns.lmplot(x="id", y="formation_energy_ev_natom", data=data)

In [ ]:
sns.lmplot(x="id", y="bandgap_energy_ev", data=data)

In [ ]:
sns.lmplot(x="lattice_vector_1_ang", y="lattice_angle_beta_degree", data=data)

In [ ]:
corr = np.corrcoef(data.values.T)
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
sns.set()
hm = sns.heatmap(corr, mask=mask, cbar=True , annot=True , square=True,
fmt='.1f', annot_kws ={'size':7})#,,cmap="YlGnBu"
#yticklabels=cols,
#xticklabels=cols)

In [ ]:
#____________________ УДАЛЯЕМ СТОЛБЦЫ ПО РЕЗУЛЬТАТАМ КОРРЕЛ. МАТРИЦЫ  _____________________
#print(type(dataX))
#dataX = dataX.drop(columns='number_of_total_atoms')
#dataX.drop(['number_of_total_atoms'], axis=1, inplace=True)
print(dataX[:1])

#X_Fin_test.drop(['number_of_total_atoms'], axis=1)


In [ ]:
#____________________ РАЗБИВКА НА ТРЕНИРОВКУ И ТЕСТ  _____________________
#   
#                   входы    выходы
#
#                     X        Y
# 75%   train      X_train  Y_train   - тренировочный набор для обучения в fit()
# 25%   test       X_test   Y_test    - тестовый набор для оценки результатов 
#
#  после обучения делаем расчет по X_test и имеем выход Y_pred. Y_pred сравниваем с Y_test и имеем оценку точности модели
#
# X - вектор входов
# Y - вектор выходов
# 
#
X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size = 0.25, shuffle = False) # а так не хочет: sk.model_selection.train_test_split , random_state=1

print("Входов: "+str(X_train.shape[1]))
print("Выходов: "+str(y_train.shape[1]))
print(y_test.describe()) # сводная инфа по таблице

#print(X_train)
# train_data = np.array(data[1:])
# print(data)

#df = pd.DataFrame(X_train)

In [ ]:
#___________________________ ПОИСК ПУСТЫХ  _______________________________
#print(df.isnull().sum()) # проверяем, есть ли пустые значения в столбцах

In [ ]:
#__________________________ НОРМИРОВКА 0 - 1 (sigm) _____________________________
# Деревья решения и Случайные леса не нуждаются в нормировке!
from sklearn.preprocessing import MinMaxScaler
sk_tr = MinMaxScaler(feature_range=(0.001, 0.999))
#X_train_norm = sk_tr.fit_transform(df)
#X_test_norm  = sk_tr.transform(X_test)
#print(X_test_norm)
#_____________________ СТАНДАРТИЗАЦИЯ -0.5 - 0 - 0.5 (tanh) ______________________
from sklearn.preprocessing import StandardScaler
#sk_tr = StandardScaler()
sk_tr.fit(X_train); X_train_std = sk_tr.transform(X_train); 
sk_tr.fit(X_test);  X_test_std  = sk_tr.transform(X_test); 
#X_train_std = X_train.copy(); X_test_std = X_test.copy()

#print("Преобразованный вход:"); print(X_test_std[:3]); print(pd.DataFrame(X_test_std).describe()) # сводная инфа по таблице
y_train_std = y_train.copy(); y_test_std = y_test.copy() #Оставляем как есть, т.к. выход "linear"
#print(y_test_std.describe())
y_train_std = y_train_std.drop(['bandgap_energy_ev'], axis=1);y_test_std = y_test_std.drop(['bandgap_energy_ev'], axis=1)

y_train2_std = y_train.copy(); y_test2_std = y_test.copy() #Оставляем как есть, т.к. выход "linear"
y_train2_std = y_train2_std.drop(['formation_energy_ev_natom'], axis=1);y_test2_std = y_test2_std.drop(['formation_energy_ev_natom'], axis=1)

#sk_tr.fit(y_train); y_train_std = sk_tr.transform(y_train); 
#sk_tr.fit(y_test);  y_test_std  = sk_tr.transform(y_test)
print("Преобразованный вЫход:"); print(y_test_std[:1]); print(pd.DataFrame(y_test_std).describe()) # сводная инфа по таблице

In [ ]:
#________________________ МОДЕЛЬ ___________________________
import theano
from keras.models       import Sequential 
from keras.layers.core  import Dense
from keras.optimizers   import SGD
from keras.layers       import Dense, Dropout, Activation

print("Входов: "+str(X_train.shape[1]))
print("Выходов: "+str(y_train_std.shape[1]))

model = Sequential()
k_init = "random_uniform" #uniform random_normal random_uniform
actForm = "tanh" #"tanh" # "sigmoid" "relu" "softmax" linear
#Softmax используется для последнего слоя глубоких нейронных сетей для задач классификации
НейроновВнутри = 50
model.add(Dense(units=НейроновВнутри, input_shape=(X_train.shape[1],), kernel_initializer=k_init, activation=actForm)) # входной слой
#model.add(Dropout(0.1))
model.add(Dense(units=НейроновВнутри, kernel_initializer=k_init, activation=actForm))               # промежуточный
#model.add(Dropout(0.1))
model.add(Dense(units=y_train_std.shape[1], kernel_initializer=k_init, activation="linear"))             # выходной слой
#model.add(Dense(input_dim=50, units=50, kernel_initializer="uniform", activation=actForm))               # промежуточный
#model.add(Dropout(0.5))
#model.summary()
#sgd = SGD(lr=0.001, decay=1e-7, momentum=.9) #SGD()
#model.compile(loss='mean_squared_logarithmic_error', optimizer='rmsprop', metrics=['mean_squared_error'])
model.compile(loss ='mean_squared_logarithmic_error', optimizer='rmsprop',metrics=['mean_squared_logarithmic_error']) #accuracy mean_squared_logarithmic_error categorical_crossentropy mean_squared_error mean_absolute_error
# СОБСТВЕННО ТРЕНИРОВКА
#model.fit(X_train_std,y_train_std,epochs=20,batch_size=10,verbose=0,validation_split=0.1) #,show_accuracy=True


In [ ]:
#________________________ ПРОВЕРКА 1 ___________________________
#y_train_predict = model.predict(X_train_std)
import sklearn.metrics as metrics

# СОБСТВЕННО ТРЕНИРОВКА
model.fit(X_train_std,y_train_std,epochs=1000,batch_size=30,verbose=0,validation_split=0.1) #,show_accuracy=True
# ПРОВЕРКА
y_test_predict = model.predict(X_test_std)
print("mean_squared_log_error: " + str(metrics.mean_squared_log_error(y_test_std, y_test_predict)))
print("r2_score 1 : " + str(metrics.r2_score(y_test_std, y_test_predict)))

score = model.evaluate(X_test_std, y_test_std, batch_size=10) #, batch_size=100

#y_test_predict = y_test_predict.astype(np.float32).reshape((-1,1))
print("1 Выход истинный:");
print(y_test_std[:10])
print("1 Выход расчитанный:");
print(y_test_predict[:10])
#print([y_test_std[:10],y_test_predict[:10],((y_test_predict[:10])*100/y_test_std[:10])])
err_per = ((y_test_predict[:10])*100/y_test_std[:10])
print("Ошибка в %:");
print(err_per)
print("Точность работы на тестовых данных: %.5f%%" % ((1-score[0])*100))
print("Test score:", score)

#print("mean_absolute_error: " + str(metrics.mean_absolute_error(y_test_std, y_test_predict)))
#print("mean_squared_error: " + str(metrics.mean_squared_error(y_test_std, y_test_predict)))
#print("r2_score: " + str(metrics.r2_score(y_test_std, y_test_predict)))
#print("mean_squared_log_error: " + str(metrics.mean_squared_log_error(y_test_std, y_test_predict)))


In [ ]:
# ---- Final TEST ---------------------------------
X_Fin_test = pd.read_csv('../input/test.csv')
ids = X_Fin_test['id']
X_Fin_test.drop(['id'], axis=1, inplace=True)
#X_Fin_test.drop(['number_of_total_atoms'], axis=1, inplace=True)
sk_tr.fit(X_Fin_test);  X_Fin_test_std  = sk_tr.transform(X_Fin_test); 
print(X_test[:1])
print(X_Fin_test[:1])

y_Fin_test_predict = model.predict(X_Fin_test_std)

In [ ]:
# ______ ВТОРОЙ ВЫХОД _______
model.fit(X_train_std,y_train2_std,epochs=100,batch_size=10,verbose=0,validation_split=0.1) #,show_accuracy=True
y_test2_predict = model.predict(X_test_std)
print("mean_squared_log_error: " + str(metrics.mean_squared_log_error(y_test2_std, y_test2_predict)))
print("r2_score 2 : " + str(metrics.r2_score(y_test2_std, y_test2_predict)))
score = model.evaluate(X_test_std, y_test2_std, batch_size=10) #, batch_size=100

print("2 Выход истинный:");
print(y_test2_std[:10])
print("2 Выход расчитанный:");
print(y_test2_predict[:10])
err_per = ((y_test2_predict[:10])*100/y_test2_std[:10])
print("Ошибка в %:");
print(err_per)
print("Точность работы на тестовых данных: %.5f%%" % ((1-score[0])*100))
print("Test score 2:", score)

#print("mean_absolute_error: " + str(metrics.mean_absolute_error(y_test2_std, y_test2_predict)))
#print("mean_squared_error: " + str(metrics.mean_squared_error(y_test2_std, y_test2_predict)))

y_Fin_test_predict2 = model.predict(X_Fin_test_std)

In [ ]:
# ---- output file compile ---------------------------------
outDF = pd.DataFrame()
outDF['id']=ids
outDF['formation_energy_ev_natom'] = y_Fin_test_predict
outDF['bandgap_energy_ev']         = y_Fin_test_predict2
print(outDF[:10])
#outDF['formation_energy_ev_natom'] = np.exp(pred1_)-1  #((np.exp(pred1)-1)*0.5 + (np.exp(pred1_)-1)*0.5) 
#outDF['bandgap_energy_ev']         = np.exp(pred2_)-1  #((np.exp(pred2)-1)*0.5 + (np.exp(pred2_)-1)*0.5)
outDF.to_csv('prediction.csv',index=False)